In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as sm
import patsy

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/Codecademy/datasets/master/streeteasy/brooklyn.csv')
bk = df[['rent', 'bedrooms',  'bathrooms', 'size_sqft', 'min_to_subway', 'building_age_yrs', 'has_washer_dryer']]
bk.head(5)

,rent,bedrooms,bathrooms,size_sqft,min_to_subway,building_age_yrs,has_washer_dryer
0,3600,3.0,2,900,4,15,0
1,3900,3.0,2,1000,4,8,0
2,2700,2.0,1,900,4,96,0
3,4900,1.0,1,1216,6,88,0
4,3900,0.0,1,1100,3,85,0


# Some Math (but don't be afraid!!)

Suppose we fit the following simple linear regression (using our eq for a line):

$\text{rent} = m*\text{size_sqft} + b + \text{error}$

This is actually shorthand for a bunch of equations, one for each row of the dataset: 

$
\begin{aligned}
3600 = m*900 + b + \text{error}_1 \\
3900 = m*1000 + b + \text{error}_2 \\
2700 = m*900 + b + \text{error}_3 \\
4900 = m*1216 + b + \text{error}_4 \\
3900 = m*1100 + b + \text{error}_5
\end{aligned}
$

Note that, for OLS regression, we want to find the values of $m$ and $b$ that minimizes the sum of all of the squared error terms:   

$\text{error}_1^2 + \text{error}_2^2 + \text{error}_3^2 + \text{error}_4^2 + \text{error}_5^2 + ...$

We can re-write the five equations above using matrices as follows: 

$
\begin{pmatrix} 3600 \\ 3900 \\2700 \\4900 \\3900 \end{pmatrix} =  m*  \begin{pmatrix} 900 \\1000 \\900 \\1216 \\1100  \end{pmatrix} + b*\begin{pmatrix} 1\\ 1\\ 1\\ 1\\ 1\\ \end{pmatrix} + \begin{pmatrix} \text{error}_1\\ \text{error}_2\\ \text{error}_3\\ \text{error}_4\\ \text{error}_5\\ \end{pmatrix}
$

Or more simply: 

$
\begin{pmatrix} 3600 \\ 3900 \\2700 \\4900 \\3900 \end{pmatrix} =
\begin{pmatrix} 1 & 900 \\ 1 & 1000 \\ 1 & 900 \\ 1 & 1216\\  1 & 1100 \end{pmatrix} \begin{pmatrix} b \\ m \end{pmatrix} +\begin{pmatrix} \text{error}_1\\ \text{error}_2\\ \text{error}_3\\ \text{error}_4\\ \text{error}_5\\ \end{pmatrix}
$

This works because of matrix multiplication: 

$
\begin{pmatrix} 1 & 900 \\ 1 & 1000 \\ 1 & 900 \\ 1 & 1216\\  1 & 1100 \end{pmatrix} * \begin{pmatrix} b \\ m \end{pmatrix} = \begin{pmatrix} b + 900m \\ b+ 1000m\\ b + 900m \\ b + 1216m \\ b + 1100m \\  \end{pmatrix} 
$

Now, we can generalize this for the purpose of multiple linear regression. For example, imagine the equation:

$rent = b_0 + b_1*\text{size_sqft} + b_2*\text{min_to_subway} + b_3*\text{has_washer_dryer} + .. + \text{error}$

In multiple linear regression, our general form of the regression equation is:

$y = b_0 + b_1x_1 + b_2x_2 + ... + b_mx_m + \text{error}$

This corresponds to many equations of the form:

$
\begin{aligned}
y_1 = b_0 + b_1x_{11} + b_2x_{21} + ... + b_mx_{m1} + \text{error}_1\\
y_2 = b_0 + b_1x_{12} + b_2x_{22} + ... + b_mx_{m2} + \text{error}_2\\
y_3 = b_0 + b_1x_{13} + b_2x_{23} + ... + b_mx_{m3} + \text{error}_3\\
. . . . . . . . . . . . . .\\
y_n = b_0 + b_1x_{1n} + b_2x_{2n} + ... + b_mx_{mn} + \text{error}_n\\
\end{aligned}
$

And we can write the shorthand for these equations as:

$
\begin{pmatrix} y_1 \\ y_2 \\ y_3 \\ . \\ . \\ .  \\ y_n \end{pmatrix}  =  \begin{pmatrix} 1 & x_{11} & x_{21} & . & . & . & x_{m1} \\1 & x_{12} & x_{22} & . & . & . & x_{m2} \\
1 & x_{13} & x_{23} & . & . & . & x_{m3} \\
                . & . & . & . & . & . & \\
                . & . & . & . & . & . & \\
                . & . & . & . & . & . & \\
                1 & x_{1n} & x_{2n} & . & . & . & x_{mn} \end{pmatrix} \begin{pmatrix} b_0 \\ b_1 \\ b_2 \\ . \\ . \\ . \\ b_m \end{pmatrix} + \begin{pmatrix} \text{error}_0 \\ \text{error}_1 \\ \text{error}_2 \\ . \\ . \\ . \\ \text{error}_n \end{pmatrix}
$


There are 4 matrices in this equation, and they each have names:

* a column matrix *y* of the outcome variable values
* the design matrix *X* which has a column of 1’s and a column for each predictor variable
* a column matrix of the intercept and slopes, often denoted by the greek letter beta ($\beta$) (this is what we want to calculate)
* the error matrix, often denoted with the greek letter epsilon ($\epsilon$)

Using these four matrices, we can write the regression equation as:

$y =  X\beta + \epsilon$

And using calculus + linear algebra, we can actually solve for the values of $\beta$ that minimize the sum of the squared values in $\epsilon$:

$\beta = (X^T X)^{-1}X^Ty$

In [39]:
# let's see this in action:

y, X = patsy.dmatrices('rent ~ building_age_yrs+ min_to_subway + has_washer_dryer', bk, return_type='dataframe')
print(X)

      Intercept  building_age_yrs  min_to_subway  has_washer_dryer
0           1.0              15.0            4.0               0.0
1           1.0               8.0            4.0               0.0
2           1.0              96.0            4.0               0.0
3           1.0              88.0            6.0               0.0
4           1.0              85.0            3.0               0.0
...         ...               ...            ...               ...
1008        1.0              88.0            5.0               0.0
1009        1.0              10.0            1.0               1.0
1010        1.0             117.0            5.0               0.0
1011        1.0               5.0            1.0               0.0
1012        1.0             116.0            3.0               1.0

[1013 rows x 4 columns]


In [40]:
model = sm.OLS(y,X)
results = model.fit()
print(results.params, '\n', results.summary().extra_txt)

Intercept           3696.170035
building_age_yrs      -5.465115
min_to_subway        -25.179742
has_washer_dryer     719.578411
dtype: float64 
 Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [41]:
# now let's convert the design matrix (X) from a dataframe to a numpy matrix
np.set_printoptions(suppress=True)
X_matrix = np.matrix(X.to_numpy())
print(X_matrix)

[[  1.  15.   4.   0.]
 [  1.   8.   4.   0.]
 [  1.  96.   4.   0.]
 ...
 [  1. 117.   5.   0.]
 [  1.   5.   1.   0.]
 [  1. 116.   3.   1.]]


In [42]:
# calculate beta using the equation above
# note that we get the same answers!
X_t = X_matrix.transpose()
beta = np.linalg.inv(X_t*X_matrix)*X_t*y.to_numpy()
print(beta)

[[3696.17003521]
 [  -5.46511521]
 [ -25.17974166]
 [ 719.5784114 ]]


## Let's see how this knowledge can help us understand errors

In [44]:
bk.loc[:,('secs_to_subway')] = bk.loc[:,('min_to_subway')]*60.0
model = sm.OLS.from_formula('rent ~ building_age_yrs + min_to_subway + secs_to_subway +has_washer_dryer', 
                            data = bk).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                   rent   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     15.46
Date:                Thu, 13 May 2021   Prob (F-statistic):           7.69e-10
Time:                        10:24:37   Log-Likelihood:                -8889.3
No. Observations:                1013   AIC:                         1.779e+04
Df Residuals:                    1009   BIC:                         1.781e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         3696.1700    105.742  

In [46]:
# similarly with categorical variables:

bk.loc[:,('no_washer_dryer')] = np.logical_not(bk.loc[:,('has_washer_dryer')])
model = sm.OLS.from_formula('rent ~ building_age_yrs + has_washer_dryer + no_washer_dryer', 
                            data = bk).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                   rent   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     21.77
Date:                Thu, 13 May 2021   Prob (F-statistic):           5.53e-10
Time:                        10:24:57   Log-Likelihood:                -8890.7
No. Observations:                1013   AIC:                         1.779e+04
Df Residuals:                    1010   BIC:                         1.780e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                2

## This knowledge can also help us figure out how to fit models using other libraries

In [55]:
# for example, scikit-learn vs. statsmodels
# note that the X matrix input looks slightly different for the two libraries

# fit model with statsmodels:
y, X = patsy.dmatrices('rent ~ building_age_yrs + has_washer_dryer', 
                       data = bk, 
                       return_type='dataframe')
print(X.head())
model_sm = sm.OLS(y,X).fit()
print(model_sm.params)

# fit model with sklearn
from sklearn.linear_model import LinearRegression
X = bk[['building_age_yrs', 'has_washer_dryer']]
y = bk[['rent']]
print(X.head())
model_skl = LinearRegression()
model_skl.fit(X, y)
print(model_skl.intercept_)
print(model_skl.coef_)

   Intercept  building_age_yrs  has_washer_dryer
0        1.0              15.0               0.0
1        1.0               8.0               0.0
2        1.0              96.0               0.0
3        1.0              88.0               0.0
4        1.0              85.0               0.0
Intercept           3583.190584
building_age_yrs      -5.608355
has_washer_dryer     746.852458
dtype: float64
   building_age_yrs  has_washer_dryer
0                15                 0
1                 8                 0
2                96                 0
3                88                 0
4                85                 0
[3583.19058388]
[[ -5.60835534 746.85245809]]
